In [31]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Question 1

In [2]:
yellow_jan_2022 = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')
yellow_jan_2022.shape


(2463931, 19)

In [6]:
yellow_jan_2022['duration'] = (yellow_jan_2022['tpep_dropoff_datetime'] - yellow_jan_2022['tpep_pickup_datetime']).apply(lambda x:x.total_seconds()/60)
yellow_jan_2022['duration']

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64

Question 2

In [7]:
yellow_jan_2022['duration'].std()

46.44530513776499

Question 3

In [12]:
((yellow_jan_2022['duration'] >= 1 ) & (yellow_jan_2022['duration'] <= 60)).mean()

0.9827547930522406

In [24]:
non_outlier_condition = ((yellow_jan_2022['duration'] >= 1 ) & (yellow_jan_2022['duration'] <= 60))
categorical = ['PULocationID','DOLocationID']
target = ['duration']


train_data = yellow_jan_2022[non_outlier_condition][categorical + target]
train_data.head()



,PULocationID,DOLocationID,duration
0,142,236,17.816667
1,236,42,8.400000
2,166,166,8.966667
3,114,68,10.033333
4,68,163,37.533333


In [41]:
train_data[categorical] = train_data[categorical].astype(str)
X_train_data_dicts = train_data[categorical].to_dict(orient = 'Records')
dv = DictVectorizer()
X_train = dv.fit_transform(X_train_data_dicts)
y_train = train_data[target].values


Question 4

In [30]:
X_train.shape

(2421440, 515)

In [33]:
lr = LinearRegression()
lr.fit(X_train,y_train)



LinearRegression()

Question 5

In [35]:
y_pred_train = lr.predict(X_train)
mean_squared_error(y_train,y_pred_train,squared=False)

6.986191065263967

In [37]:
def read_df(filename):
    df = pd.read_parquet(filename)
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).apply(lambda x:x.total_seconds()/60)
    non_outlier_condition = ((df['duration'] >= 1 ) & (df['duration'] <= 60))
    df = df[non_outlier_condition]
    return df
    


    

In [38]:
valid_data = read_df('../data/yellow_tripdata_2022-02.parquet')
valid_data.shape

(2918187, 20)

In [39]:
valid_data[categorical] = valid_data[categorical].astype(str)
X_valid_data_dicts = valid_data[categorical].to_dict(orient = 'Records')
X_valid = dv.transform(X_valid_data_dicts)
y_valid= valid_data[target].values

Question 6

In [40]:
y_pred_valid = lr.predict(X_valid)
mean_squared_error(y_valid,y_pred_valid,squared=False)

7.786413838484518